In [3]:
#!pip install nltk
import sys
from IPython.display import HTML
sys.path.append('/tmp/dd10b84e-1fb0-4dcd-b55d-79d2aa0d553f/lib/python2.7/site-packages/')

# !pip install spacy
# !python -m spacy.en.download all
# !pip install nltk
# !pip install gensim

In [1]:
import nltk
nltk.download('reuters')
corpus = nltk.corpus.reuters
sents = [map(unicode, tokens) for tokens in corpus.sents()]

Model already installed. Please run 'python -m spacy.en.download --force' to reinstall.


In [4]:
import spacy
from spacy.matcher import Matcher
from spacy.attrs import ORTH, IS_PUNCT
nlp = spacy.en.English(parse = False)

def merge_phrases(matcher, doc, i, matches):
    '''
    Merge a phrase. We have to be careful here because we'll change the token indices.
    To avoid problems, merge all the phrases once we're called on the last match.
    '''
    if i != len(matches)-1:
        return None
    # Get Span objects
    spans = [(ent_id, label, doc[start : end]) for ent_id, label, start, end in matches]
    for ent_id, label, span in spans:
        span.merge(label=label, tag='NNP' if label else span.root.tag_)
        

matcher = Matcher(nlp.vocab)
matcher.add_entity('USA', on_match = merge_phrases)
matcher.add_pattern("USA", [{ORTH: "U"}, {IS_PUNCT: True}, {ORTH: "S"}, {IS_PUNCT:True}])


In [8]:
from spacy.tokens import Doc

def process_token(token):
    if token.is_punct or token.is_space:
        return False
    elif token.like_url:
        return'URL'
    elif token.like_email:
        return'EMAIL'
    elif token.like_num:
        return'NUM'
    else:
        return token.lower_.replace(" ",'')
    
    
def process_sentence(tokenized_sent):
    tokens = []
    
    doc = Doc(nlp.vocab, words = tokenized_sent)
    nlp.tagger(doc)
    matcher(doc)
    
    for token in doc:
        processed_token = process_token(token)
        if processed_token:
            tokens.append(processed_token)    
    return tokens

In [34]:
processed_sents = map(process_sentence, sents)

u'u . s .'

In [50]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=processed_sents, ###tokenized senteces, list of list of strings
                 size=300,  #size of embedding vectors
                 workers=8, #how many threads?
                 min_count=5, #minimum number of token instances to be considered
                 sample=0, #weight of downsampling common words? 
                 sg = 0, #should we use skip-gram? if 0, then cbow
                 hs=0, #heirarchical softmax?
                 iter=5 #training epocs
        )

In [53]:
import bz2

#write to disk
vector_file = 'custom-vectors.word2vec'
model.save_word2vec_format(str(vector_file), binary=False)


#delete first 2 lines; contains non-decodeable stuff
with open(str(vector_file)) as f:
    data = f.readlines()
f.close()
with open(str(vector_file), 'wb') as f:
    f.writelines(data[2:])
    

#compress via bz2
compressed_filename = vector_file + '.bz2'
z = bz2.compress(open(vector_file).read())
with open(compressed_filename, 'w+') as out:
    out.write(z)

    
#write to binary with spacy    
spacy.vocab.write_binary_vectors(compressed_filename,'custom-vectors.bin')

In [54]:
import spacy
from spacy.vocab import Vocab
v = Vocab()

nlp.vocab.load_vectors_from_bin_loc(str(vector_file))

/usr/local/lib/python2.7/dist-packages/gensim/models/word2vec.py:459: UserWarning: C extension compilation failed, training will be slow. Install a C compiler and reinstall gensim for fast training.
  warnings.warn("C extension compilation failed, training will be slow. Install a C compiler and reinstall gensim for fast training.")
